<a href="https://colab.research.google.com/github/rahavi-r31/CollegeWorks-ML-/blob/main/Hierarchical_Clustering_on_Heart_Disease_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pgmpy.models
import pgmpy.estimators

data = pd.read_csv('heart_disease.csv')
from sklearn.preprocessing import StandardScaler

# Remove missing data
data = data.dropna()

# Split the data into features and target
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
from scipy.cluster.hierarchy import linkage, fcluster

# Perform hierarchical clustering
Z = linkage(X, method='ward')

# Determine the number of clusters
max_d = 20
clusters = fcluster(Z, max_d, criterion='distance')
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination

# Define the Bayesian network
model = BayesianNetwork([('age', 'trestbps'), ('age', 'chol'), ('sex', 'trestbps'),
                       ('cp', 'chol'), ('fbs', 'restecg'), ('restecg', 'thalach'),
                       ('thalach', 'exang'), ('exang', 'num'), ('oldpeak', 'slope'),
                       ('num', 'diagnosis')])

# Fit the model
model.fit(data, estimator=MaximumLikelihoodEstimator)
# Train the model
model.fit(data, estimator=BayesianEstimator)
infer = VariableElimination(model)
q = infer.query(['diagnosis'], evidence={'age': 48, 'sex': 1, 'cp': 2, 'trestbps': 120,
                                          'chol': 233, 'fbs': 0, 'restecg': 1, 'thalach': 163,
                                          'exang': 1, 'oldpeak': 0.8, 'slope': 2})
print(q)

# Diagnose a heart patient
patient_data = {'age': 50, 'sex': 1, 'cp': 2, 'trestbps': 130, 'chol':246, 'fbs': 0, 'restecg': 1, 'thalach': 168,
                'exang': 0, 'oldpeak': 1.0, 'slope': 2}


+--------------+------------------+
| diagnosis    |   phi(diagnosis) |
+==============+==================+
| diagnosis(0) |           0.3393 |
+--------------+------------------+
| diagnosis(1) |           0.6607 |
+--------------+------------------+
